In [1]:
import tensorflow as tf
import pandas as pd
tf.__version__

'2.13.0'

In [3]:
# Load training dataset
X_train = pd.read_csv("../input_data/CSVs/dota2Train.csv", header=None)
# Set the training target as the first column (win condition) and remove it from the features to train from 
y_train = X_train.pop(0)

# Download a pretrained model for this task and save it locally (note - URL in book is wrong, this is fixed)
model_path = tf.keras.utils.get_file('exercise5_01_model.h5', 'https://github.com/PacktWorkshops/The-TensorFlow-Workshop/blob/master/Chapter05/model/exercise5_01_model.h5?raw=true')
# Load the model
model = tf.keras.models.load_model(model_path)
# Display the layout of the model
model.summary()

4904824/4904824 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 512)               59904     
                                                                 
 dense_6 (Dense)             (None, 512)               262656    
                                                                 
 dense_7 (Dense)             (None, 128)               65664     
                                                                 
 dense_8 (Dense)             (None, 128)               16512     
                                                                 
 dense_9 (Dense)             (None, 1)                 129       
                                                                 
Total params: 404865 (1.54 MB)
Trainable params: 404865 (1.54 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [4]:
# Load and prep the testing dataset
X_test = pd.read_csv("../input_data/CSVs/dota2Test.csv", header=None)
y_test = X_test.pop(0)
# Generate predictions for the testing dataset and print the last 5
preds_proba = model.predict(X_test)
preds_proba[:5]

322/322 [==============================] - 2s 2ms/step


array([[0.4366297 ],
       [0.52438205],
       [0.6132363 ],
       [0.6282078 ],
       [0.3523952 ]], dtype=float32)

In [5]:
# Covert the predictions to booleans based on whether the certainty of the prediction being 1 is 50% or more
preds = preds_proba >= 0.5
preds[:5]

array([[False],
       [ True],
       [ True],
       [ True],
       [False]])

In [7]:
# Load different accuracy metrics to evaluate the model on
acc = tf.keras.metrics.Accuracy()
prec = tf.keras.metrics.Precision()
rec = tf.keras.metrics.Recall()

# Replace negative outcomes of `-1` with `0` to match the model's output format
y_test = y_test.replace(-1,0)

# Process accuracy using the predictions and actual values as input
acc.update_state(preds, y_test)
acc_results = acc.result().numpy()
acc_results

0.596464

In [9]:
# Calculate precision score
prec.update_state(preds, y_test)
prec_results = prec.result().numpy()
prec_results

0.59578335

In [8]:
# Calculate recall score
rec.update_state(preds, y_test)
rec_results = rec.result().numpy()
rec_results

0.6294163

In [10]:
# Calculate F1 score on the dataset
f1 = 2*(prec_results * rec_results) / (prec_results + rec_results)
f1

0.6121381493171637